https://eaton-lab.org/slides/genomics/answers/nb-10.2-de-Bruijn.html

In [1]:
import random
import toyplot

### Inferring the De bruijn graph

In [25]:
def get_kmer_count_from_sequence(sequence, k=3, cyclic=True):
    # dict to store kmers
    kmers = {}
    # count how many times each occurred in this sequence (treated as cyclic)
    for i in range(0, len(sequence)):
        kmer = sequence[i:i + k]
        # for cyclic sequence get kmers that wrap from end to beginning
        length = len(kmer)
        if cyclic:
            if len(kmer) != k:
                kmer += sequence[:(k - length)]
        # if not cyclic then skip kmers at end of sequence
        else:
            if len(kmer) != k:
                continue
        # count occurrence of this kmer in sequence
        if kmer in kmers:
            kmers[kmer] += 1
        else:
            kmers[kmer] = 1
    return kmers

### Test case 1

In [26]:
Alphabet = "anupaisagenius"
kmers = get_kmer_count_from_sequence(Alphabet, k=4, cyclic=False)
kmers

{'anup': 1,
 'nupa': 1,
 'upai': 1,
 'pais': 1,
 'aisa': 1,
 'isag': 1,
 'sage': 1,
 'agen': 1,
 'geni': 1,
 'eniu': 1,
 'nius': 1}

### Building the de-Bruijn graph

In [27]:
import hashlib
class DeBruijnGraph:
    def _init_(self, k):
        self.k = k
        self.edges = {}

    def add_kmer(self, kmer):
        prefix = kmer[:-1]
        suffix = kmer[1:]
        prefix_hash = hashlib.sha256(prefix.encode()).hexdigest()
        if prefix_hash in self.edges:
            self.edges[prefix_hash].append(suffix)
        else:
            self.edges[prefix_hash] = [suffix]

    def get_edges(self, prefix):
        prefix_hash = hashlib.sha256(prefix.encode()).hexdigest()
        if prefix_hash in self.edges:
            return self.edges[prefix_hash]
        else:
            return []

def create_debruijn_from_string(text, k):
    graph = DeBruijnGraph(k)
    for i in range(len(text) - k + 1):
        kmer = text[i:i+k]
        graph.add_kmer(kmer)
    return graph

In [28]:
def get_debruijn_edges_from_kmers(kmers):
    # store edges as tuples in a set
    edges = set()
    # compare each (k-1)mer
    for k1 in kmers:
        for k2 in kmers:
            if k1 != k2:            
                # if they overlap then add to edges
                if k1[1:] == k2[:-1]:
                    edges.add((k1[:-1], k2[:-1]))
                if k1[:-1] == k2[1:]:
                    edges.add((k2[:-1], k1[:-1]))
    return edges

### Test case 2

In [29]:
Alphabets = 'ButIgotSmarter'
kmers = get_kmer_count_from_sequence(Alphabets, k=4, cyclic=True)
print(kmers)
edges = get_debruijn_edges_from_kmers(kmers)
edges

{'ButI': 1, 'utIg': 1, 'tIgo': 1, 'Igot': 1, 'gotS': 1, 'otSm': 1, 'tSma': 1, 'Smar': 1, 'mart': 1, 'arte': 1, 'rter': 1, 'terB': 1, 'erBu': 1, 'rBut': 1}


{('But', 'utI'),
 ('Igo', 'got'),
 ('Sma', 'mar'),
 ('art', 'rte'),
 ('erB', 'rBu'),
 ('got', 'otS'),
 ('mar', 'art'),
 ('otS', 'tSm'),
 ('rBu', 'But'),
 ('rte', 'ter'),
 ('tIg', 'Igo'),
 ('tSm', 'Sma'),
 ('ter', 'erB'),
 ('utI', 'tIg')}

### Ploting the graph

In [30]:
def plot_debruijn_graph(edges, width=500, height=500):
    "returns a toyplot graph from an input of edges"
    graph = toyplot.graph(
        [i[0] for i in edges],
        [i[1] for i in edges],
        width=width,
        height=height,
        tmarker=">", 
        vsize=25,
        vstyle={"stroke": "white", "stroke-width": 2, "fill": "none"},
        vlstyle={"font-size": "11px"},
        estyle={"stroke": "black", "stroke-width": 2},
        layout=toyplot.layout.FruchtermanReingold(edges=toyplot.layout.CurvedEdges()))
    return graph

### With sequence as Example (Test case 3)

In [31]:
def random_sequence(seqlen):
    "Generate a random DNA sequence of a given length "
    return "".join([random.choice("ACGT") for i in range(seqlen)])

In [32]:
random.seed(123)
genome1 = random_sequence(35)
genome1

'AGATGAATGGACCGGCCATATAAGTAAACCAGTTG'

In [33]:
kmers = get_kmer_count_from_sequence(genome1, k=3)
kmers

{'AGA': 1,
 'GAT': 1,
 'ATG': 2,
 'TGA': 2,
 'GAA': 1,
 'AAT': 1,
 'TGG': 1,
 'GGA': 1,
 'GAC': 1,
 'ACC': 2,
 'CCG': 1,
 'CGG': 1,
 'GGC': 1,
 'GCC': 1,
 'CCA': 2,
 'CAT': 1,
 'ATA': 2,
 'TAT': 1,
 'TAA': 2,
 'AAG': 1,
 'AGT': 2,
 'GTA': 1,
 'AAA': 1,
 'AAC': 1,
 'CAG': 1,
 'GTT': 1,
 'TTG': 1,
 'GAG': 1}

In [34]:
edges = get_debruijn_edges_from_kmers(kmers)
edges

{('AA', 'AA'),
 ('AA', 'AC'),
 ('AA', 'AG'),
 ('AA', 'AT'),
 ('AC', 'CC'),
 ('AG', 'GA'),
 ('AG', 'GT'),
 ('AT', 'TA'),
 ('AT', 'TG'),
 ('CA', 'AG'),
 ('CA', 'AT'),
 ('CC', 'CA'),
 ('CC', 'CG'),
 ('CG', 'GG'),
 ('GA', 'AA'),
 ('GA', 'AC'),
 ('GA', 'AG'),
 ('GA', 'AT'),
 ('GC', 'CC'),
 ('GG', 'GA'),
 ('GG', 'GC'),
 ('GT', 'TA'),
 ('GT', 'TT'),
 ('TA', 'AA'),
 ('TA', 'AT'),
 ('TG', 'GA'),
 ('TG', 'GG'),
 ('TT', 'TG')}

In [35]:
kmers = get_kmer_count_from_sequence(genome1, k=6, cyclic=True)
edges = get_debruijn_edges_from_kmers(kmers)
print("the true sequence: {}".format(genome1))
plot_debruijn_graph(edges, width=600, height=400)

the true sequence: AGATGAATGGACCGGCCATATAAGTAAACCAGTTG


(<toyplot.canvas.Canvas at 0x2cbf33d4fa0>,
 <toyplot.mark.Graph at 0x2cbfb8bad10>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="400.0px" viewBox="0 0 600.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="td70bca8a2684483bb3152dc45697b4ca"> AAACC AACCA AAGTA AATGG ACCAG ACCGG AGATG AGTAA AGTTG ATAAG ATATA ATGAA ATGGA CAGTT CATAT CCAGT CCATA CCGGC CGGCC GAATG GACCG GAGAT GATGA GCCAT GGACC GGCCA GTAAA GTTGA TAAAC TAAGT TATAA TGAAT TGAGA TGGAC TTGAG

In [36]:
kmers = get_kmer_count_from_sequence(genome1, k=6, cyclic=False)
edges = get_debruijn_edges_from_kmers(kmers)
plot_debruijn_graph(edges, width=800, height=400);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="800.0px" height="400.0px" viewBox="0 0 800.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t9c5df8b98f714ac2875879f052457778"> AAACC AACCA AAGTA AATGG ACCAG ACCGG AGATG AGTAA ATAAG ATATA ATGAA ATGGA CAGTT CATAT CCAGT CCATA CCGGC CGGCC GAATG GACCG GATGA GCCAT GGACC GGCCA GTAAA TAAAC TAAGT TATAA TGAAT TGGAC

In [39]:
import collections
def create_debruijn_graph(kmers):
    graph = collections.defaultdict(list)
    for kmer in kmers:
        graph[kmer[:-1]].append(kmer[1:])
    return graph
kmers = get_kmer_count_from_sequence("AGATGAATGGACCGGCCATATAAGTAAACCAGTTG", 3, True)
debruijn_graph = create_debruijn_graph(kmers)
print(debruijn_graph)
edges = get_debruijn_edges_from_kmers(kmers)
print("the true sequence: {}".format(genome1))
plot_debruijn_graph(edges, width=600, height=400)

defaultdict(<class 'list'>, {'AG': ['GA', 'GT'], 'GA': ['AT', 'AA', 'AC', 'AG'], 'AT': ['TG', 'TA'], 'TG': ['GA', 'GG'], 'AA': ['AT', 'AG', 'AA', 'AC'], 'GG': ['GA', 'GC'], 'AC': ['CC'], 'CC': ['CG', 'CA'], 'CG': ['GG'], 'GC': ['CC'], 'CA': ['AT', 'AG'], 'TA': ['AT', 'AA'], 'GT': ['TA', 'TT'], 'TT': ['TG']})
the true sequence: AGATGAATGGACCGGCCATATAAGTAAACCAGTTG


(<toyplot.canvas.Canvas at 0x2cbfb8b90f0>,
 <toyplot.mark.Graph at 0x2cbfb85b280>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="400.0px" viewBox="0 0 600.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t3e181d13aa48454db39b8b08b2077ec8"> AA AC AG AT CA CC CG GA GC GG GT TA TG TT